In [ ]:
import numpy as np
import pandas as pd
import os,random,time
from sklearn.model_selection import train_test_split
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import json
import matplotlib.pyplot  as plt                   
import cv2
import tensorflow as tf

from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

In [ ]:
import json, codecs
with codecs.open("../input/herbarium-2020-fgvc7/nybg2020/train/metadata.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
    
with codecs.open("../input/herbarium-2020-fgvc7/nybg2020/test/metadata.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    test_meta = json.load(f)

In [ ]:
train_meta.keys()

In [ ]:
train_meta

In [ ]:
sample_sub = pd.read_csv('../input/herbarium-2020-fgvc7/sample_submission.csv')
display(sample_sub)

In [ ]:
train_df=pd.DataFrame(train_meta['annotations'])
train_df.head()

In [ ]:
train_cat=pd.DataFrame(train_meta['categories'])
train_cat.columns = ['family', 'genus', 'category_id', 'category_name']
train_cat.head()

In [ ]:
train_img=pd.DataFrame(train_meta['images'])
train_img.columns = ['file_name', 'height', 'image_id', 'license', 'width']
train_img.head()

In [ ]:
train_reg = pd.DataFrame(train_meta['regions'])
train_reg.columns = ['region_id', 'region_name']
train_reg.head()

Merge DataFrames

In [ ]:
train_df = train_df.merge(train_cat, on='category_id', how='outer')
train_df = train_df.merge(train_img, on='image_id', how='outer')
train_df = train_df.merge(train_reg, on='region_id', how='outer')

In [ ]:
train_df.head()

In [ ]:
na = train_df.file_name.isna()
keep = [x for x in range(train_df.shape[0]) if not na[x]]
train_df = train_df.iloc[keep]

In [ ]:
dtypes = ['int32', 'int32', 'int32', 'int32', 'object', 'object', 'object', 'object', 'int32', 'int32', 'int32', 'object']
for n, col in enumerate(train_df.columns):
    train_df[col] = train_df[col].astype(dtypes[n])
print(train_df.info())
display(train_df)

In [ ]:
test_df = pd.DataFrame(test_meta['images'])
test_df.columns = ['file_name', 'height', 'image_id', 'license', 'width']
print(test_df.info())
display(test_df)

In [ ]:
train_df.to_csv('full_train_data.csv', index=False)
test_df.to_csv('full_test_data.csv', index=False)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Let's Create Model!

In [ ]:
train=train_df[['file_name','family','genus','category_id']][:40000]
test=test_df['file_name']
train.head()

In [ ]:
TRAIN_PATH='../input/herbarium-2020-fgvc7/nybg2020/train/'
TEST_PATH='../input/herbarium-2020-fgvc7/nybg2020/test/'
def train_path(st):
    return TRAIN_PATH + st
def test_path(st):
    return TEST_PATH + st

In [ ]:
x_train=train['file_name']
x_train=x_train.apply(train_path).values
test=test.apply(test_path).values
y_train_f=train['family']
y_train_g=train['genus']
y_train_c=train['category_id']
x_train,x_valid,y_train_f,y_valid_f=train_test_split(x_train,y_train_f,test_size=0.1,random_state=2020)

In [ ]:
def decode_image(filename, label=None, image_size=(120, 120)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
#     image = tf.image.adjust_brightness(image, delta=0.2)
#     image = tf.image.adjust_contrast(image,2)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train_f))
    .map(decode_image)
    .cache()
    .map(data_augment)
    .repeat()
    .shuffle(512)
    .batch(20)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid_f))
    .map(decode_image)
    .batch(20)
    .cache()
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test)
    .map(decode_image)
    .batch(20)
)

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

ch_p = ModelCheckpoint(filepath="model_ir.h5", monitor='val_loss', save_weights_only=True,
                                                 verbose=1)

In [ ]:
model = tf.keras.Sequential([
        tf.keras.applications.InceptionResNetV2(
            input_shape=(120, 120, 3),
            weights='imagenet',
            include_top=False
        ),
        L.GlobalAveragePooling2D(),
        L.Dense(y_train_f.shape[0], activation='softmax')
    ])
        
model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['categorical_accuracy']
    )
#     model.summary()

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
STEPS_PER_EPOCH = x_train.shape[0] // 20 

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=40, 
    callbacks=[lr_schedule, ch_p],
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=valid_dataset
)